In [7]:
import pymc as pm
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image
#import utils
%matplotlib inline

# the same as soft evidence fact
f_1_of_10000 = pm.Categorical("f_1_of_10000", [0.1, 0.9])
print(f_1_of_10000.value)
prior_HIV_positive = pm.Categorical("I_am_HIV_positive", [0.9999, 0.0001])
I_dont_know = pm.Categorical("I_dont_know", [0.5, 0.5])

@pm.deterministic
def p_I_am_HIV_positive_soft(f_1_of_10000=f_1_of_10000, prior_HIV_positive=prior_HIV_positive, I_dont_know=I_dont_know):
    if (f_1_of_10000 == 1):
        return prior_HIV_positive
    else:
        return I_dont_know

print(p_I_am_HIV_positive_soft.value)

@pm.stochastic(observed=True)
def HIV_test_false_positive(value=1, p_I_am_HIV_positive_soft=p_I_am_HIV_positive_soft):
    if ( p_I_am_HIV_positive_soft != 0 and p_I_am_HIV_positive_soft != 1 ):
        return -np.inf
    if ( value != 0 and value != 1 ):
        return -np.inf
    
    if ( p_I_am_HIV_positive_soft == 0 ):
        prob = 0.001 if value == 1 else 0.999 # 0.001 - prob that test is true if a am not really HIV
    else:
        prob = 0.999 if value == 1 else 0.001 # 0.999 - prob that test is true if a am really HIV
    return np.log(prob)
     
model = pm.Model([f_1_of_10000, prior_HIV_positive, I_dont_know, p_I_am_HIV_positive_soft, HIV_test_false_positive])
#graph = pm.graph.graph(model)
#Image(graph.create_png())
mcmc = pm.MCMC(model)
mcmc.sample(20000, 2000)

alpha_samples = mcmc.trace("p_I_am_HIV_positive_soft")[:]

print(sum(alpha_samples))
# f_HIV_test_false_positive = pm.math.switch(
#     p_I_am_HIV_positive_soft == 0,
#     pm.Categorical("test_if_I_am_not_HIV", [0.999, 0.001]),
#     pm.Categorical("test_if_I_am_HIV", [0.001, 0.999]),
#     observed=1
# )

1
0
 [-----------------100%-----------------] 20000 of 20000 complete in 32.5 sec17711


In [6]:
# The parameters to be inferred. We only know them here because we are synthesising the data.
true_alpha = 10
true_beta = 50

num_flashes = 50

# Generate the angles
true_thetas = np.random.uniform(low=-0.5*np.pi, high=0.5*np.pi, size=num_flashes)

# Generate the x coordinates of the flashes along the coastline
data = true_alpha + true_beta * np.tan(true_thetas)
print(data)

[  34.48812604   60.70183575 -395.06191324  -13.26777975   87.06211973
  238.51682684 -769.73061982    5.27941371   -1.48011857  -50.17412454
    0.97595069   26.42491173  -84.3494807    11.11215006   63.81445975
   30.11792832   -9.94067439   61.54219083 -153.43098829  -17.11648649
   37.1276701    26.63062263 -119.26462687  -45.40224821   -1.08828223
   18.05540347  117.50872415   45.1306907   -12.60321802   35.86636911
   13.03566077  104.7065462  -572.25351675   29.95359632  -11.88847627
    7.31837546  139.43694805   13.57138393  531.77325519   55.58575935
   -7.47213901   75.42960648  -26.83676874   56.68983289   26.94977381
   12.53647425 -176.29622473  103.71966544 -222.9610755    65.73200885]


In [ ]:
def age_logp(age):
    if age < 0 or age > 110:
        return -np.inf
    elif 0 <= age < 20:
        prob = age_dist['0-20']
    elif 20 <= age < 40:
        prob = age_dist['20-40']
    elif 40 <= age < 60:
        prob = age_dist['40-60']
    elif 60 <= age < 80:
        prob = age_dist['60-80']
    elif age >= 80:
        prob = age_dist['80-inf']
    return np.log(prob)


@pymc.stochastic
def age(value=0):
    def logp(value):
        return age_logp(value)